In [1]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [7]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
import cassio

In [8]:
!pip install pyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.7 MB/s eta 0:00:00


In [10]:
from PyPDF2 import PdfReader

In [11]:
ASTRA_DB_APPLICATION_TOKEN = "YOUR_ASTRA_DB_TOKEN"
ASTRA_DB_ID="YOUR_ASTRA_DB_ID"
OPENAI_API_KEY="YOUR_OPENAI_API_KEY"

In [12]:
pdfreader = PdfReader('Climate-report.pdf')

In [13]:
from typing_extensions import Concatenate

raw_text =''

for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

In [14]:
raw_text

'State of the Global Climate  \n2023\nWMO-No. 1347WEATHER CLIMATE WATER\nSUBJECT TO  \nCOPY EDITBWMO-No. 1347\n© World Meteorological Organization, 2024\nThe right of publication in print, electronic and any other form and in any language is reserved by WMO.  \nShort extracts from WMO publications may be reproduced without authorization, provided that the \ncomplete source is clearly indicated. Editorial correspondence and requests to publish, reproduce \nor translate this publication in part or in whole should be addressed to:\nChair, Publications Board\nWorld Meteorological Organization (WMO)\n7 bis, avenue de la Paix Tel.: +41 (0) 22 730 84 03\nP.O. Box 2300 Fax: +41 (0) 22 730 81 17\nCH-1211 Geneva 2, Switzerland Email: publications@wmo.int\nISBN 978-92-63-11347-4 \nCover illustration: A melting glacier with chunks of ice breaking apart. The background captures the consequences of global \nwarming on ice formations. Generative AI. Par Regina. N° de fichier:  640624130\nNOTE\nThe de

In [15]:
##Initialize connection to DB

cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <LibevConnection(140058815863088) 51c4e4e7-e369-4422-ba62-bf2fe2c43763-us-east1.db.astra.datastax.com:29042:c3a97259-a78e-42a4-89e4-111a1ad07631> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [19]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [20]:
#Create vector store

astra_vector_store = Cassandra(
    embedding=embedding,
    table_name='pdfqueryproj',
    session=None,
    keyspace=None
)

In [21]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)

texts = text_splitter.split_text(raw_text)

In [24]:
#Load dataset in vector store

astra_vector_store.add_texts(texts[:50])

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [26]:
first_question = True

while True:
  if first_question:
    query_text  =input("Enter your question..(type quit to exit): ").strip()
  else:
    query_text=input("\n What is your next question.. :").strip()

  if query_text.lower() =="quit":
    break
  if query_text.lower() == "":
    continue
  first_question = False

  print("\nQuestion: \"%s\"" % query_text)
  answer = astra_vector_index.query(query_text, llm=llm).strip()
  print("\nAnswer: \"%s\"\n" % answer)

  print("FIRST DOCUMENTS BY RELEVANCE: ")
  for doc, score in astra_vector_store.similarity_search_with_score(query_text,k=4):
    print("  [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))

Enter your question..(type quit to exit): What is state of glacier?
nQuestion: "What is state of glacier?"



Answer: \The preliminary data from the global set of reference glaciers for the hydrological year 2022-2023 show they experienced the largest loss of ice on record (1950–2023), driven by extremely negative mass balance in both western North America and Europe. Glaciers in Switzerland lost around 10% of their remaining volume in the past two years."

FIRST DOCUMENTS BY RELEVANCE: 


  [0.8935] "7 bis, avenue de la Paix Tel.: +41 (0) 22 730 84 03
P.O. Box 2300 Fax: +41 (0) 22 73 ..."
  [0.8929] "dioxide, methane, and nitrous oxide – reached record high 
observed levels.
Global m ..."
  [0.8917] "State of the Global Climate  
2023
WMO-No. 1347WEATHER CLIMATE WATER
SUBJECT TO  
CO ..."
  [0.8863] "(1950–2023), driven by extremely negative mass 
balance in both western North Americ ..."

 What is your next question.. :quit
